In [1]:
import pandas as pd
import numpy as np
import sys
import json 
import random
import math
import collections
import seaborn as sns
import networkx as nx

## Load Data

In [2]:
G = nx.read_weighted_edgelist('../data/conn_communities.weighted.edgelist')
G = nx.relabel_nodes(G, {node:int(node) for node in G.nodes}) 
df = pd.read_csv('../data/conn_communities.csv',index_col=0)

df.head()

,index,position,name,city,menu,link,score,N_reviews,price,Name,...,specialDiets,positionlink,province,Link,provinceOK,louvain,kmeans,async_fluid_7,multicore,demon
0,0,2018,Osteria al Borgo,Lucca,False,/Restaurant_Review-g187898-d23137800-Reviews-O...,5.0,6,NaN,Osteria al Borgo,...,"Vegetarian Friendly, Vegan Options, Gluten Fre...",j39_https://maps.google.com/maps?saddr=&daddr=...,pisa,/Restaurant_Review-g187898-d23137800-Reviews-O...,0,8.0,1.0,6.0,2.0,58
1,12,1309,Il Genio Italiano,Florence,True,/Restaurant_Review-g187895-d2483611-Reviews-Il...,4.0,229,2.5,Il Genio Italiano,...,Vegetarian Friendly,vs1_https://maps.google.com/maps?saddr=&daddr=...,florence,/Restaurant_Review-g187895-d2483611-Reviews-Il...,1,8.0,0.0,6.0,2.0,58
2,37,1316,La Taverna,Scandicci,False,/Restaurant_Review-g644284-d2660933-Reviews-La...,4.0,236,2.5,La Taverna,...,Vegetarian Friendly,zhJ_https://maps.google.com/maps?saddr=&daddr=...,florence,/Restaurant_Review-g644284-d2660933-Reviews-La...,1,8.0,0.0,6.0,2.0,58
3,48,1301,Bar Pasticceria Deanna,Florence,False,/Restaurant_Review-g187895-d2077023-Reviews-Ba...,4.0,162,1.0,Bar Pasticceria Deanna,...,"Vegetarian Friendly, Vegan Options, Gluten Fre...",hSR_https://maps.google.com/maps?saddr=&daddr=...,florence,/Restaurant_Review-g187895-d2077023-Reviews-Ba...,1,8.0,0.0,6.0,3.0,58
4,50,1332,Perseus Fiesolano,Fiesole,True,/Restaurant_Review-g187896-d2720609-Reviews-Pe...,3.5,754,2.5,Perseus Fiesolano,...,Gluten Free Options,d5H_https://maps.google.com/maps?saddr=&daddr=...,florence,/Restaurant_Review-g187896-d2720609-Reviews-Pe...,1,8.0,0.0,6.0,3.0,58


In [25]:
True + True

2

In [24]:
df.columns

Index(['index', 'position', 'name', 'city', 'menu', 'link', 'score',
       'N_reviews', 'price', 'Name', 'Province', 'City', 'claimed',
       'covidMeasure', 'travellersChoice', 'latitude', 'longitude', 'cuisines',
       'meals', 'specialDiets', 'positionlink', 'province', 'Link',
       'provinceOK', 'louvain', 'kmeans', 'async_fluid_7', 'multicore',
       'demon'],
      dtype='object')

In [17]:
dfg = pd.read_csv('../data/conn_communities.weighted.edgelist',index_col = None, header = None, sep=' ', names=['u','v','weight'])
dfg

,u,v,weight
0,0,1,1.0
1,0,2,1.0
2,0,3,1.0
3,0,4,1.0
4,0,5,1.0
...,...,...,...
136641,14067,14999,1.0
136642,14067,15000,2.0
136643,14999,15000,1.0
136644,14669,14892,1.0


In [18]:
C = G.copy()
nodeList = list(C.nodes)

In [20]:
noedge = []

while len(noedge) < 136646:
    
    u = random.choice(nodeList)
    v = random.choice(nodeList)
    
    if u != v and C.has_edge(u,v) == False:
        
        C.add_edge(u,v, weight=0)
        dfg = dfg.append({'u':u,'v':v,'weight':0},ignore_index=True)
        noedge.append((u,v))

In [21]:
dfg.to_csv('../data/dfg.csv')

In [22]:
len(noedge)

136646

In [55]:
dfg = pd.read_csv('../data/dfg.csv',index_col = 0)

In [56]:
dfg

,u,v,weight
0,0,1,1.0
1,0,2,1.0
2,0,3,1.0
3,0,4,1.0
4,0,5,1.0
...,...,...,...
273287,7721,6677,0.0
273288,3059,8949,0.0
273289,13962,4064,0.0
273290,2093,4084,0.0


In [57]:
df.isnull().sum()

index                  0
position               0
name                   0
city                   0
menu                   0
link                   0
score                518
N_reviews              0
price               2892
Name                   0
Province               0
City                   0
claimed                0
covidMeasure           0
travellersChoice       0
latitude               0
longitude              0
cuisines            1337
meals               6281
specialDiets        8389
positionlink           1
province               0
Link                1752
provinceOK             0
louvain                0
kmeans                 0
async_fluid_7          0
multicore              0
demon               5730
dtype: int64

In [58]:
df.loc[df.city.isna(),'city'] = df.loc[df.city.isna(),'province']

In [59]:
df.loc[df.specialDiets.isna(),'specialDiets'] = 0
df.loc[df.cuisines.isna(),'cuisines'] = 0

In [60]:
df.loc[df.score.isna(),'score'] = 0
df.loc[df.price.isna(),'price'] = 0

In [61]:
df.isnull().sum()

index                  0
position               0
name                   0
city                   0
menu                   0
link                   0
score                  0
N_reviews              0
price                  0
Name                   0
Province               0
City                   0
claimed                0
covidMeasure           0
travellersChoice       0
latitude               0
longitude              0
cuisines               0
meals               6281
specialDiets           0
positionlink           1
province               0
Link                1752
provinceOK             0
louvain                0
kmeans                 0
async_fluid_7          0
multicore              0
demon               5730
dtype: int64

In [66]:
for i in dfg.index:
    
    u = dfg.loc[i,'u'] 
    v = dfg.loc[i,'v']
    
    if df.loc[u,'province'] == df.loc[v,'province']: dfg.loc[i,'province'] = 1
    else: dfg.loc[i,'province'] = 0
    if df.loc[u,'city'] == df.loc[v,'city']: dfg.loc[i,'city'] = 1
    else: dfg.loc[i,'city'] = 0
    
    xu = df.loc[u,'longitude']
    xv = df.loc[v,'longitude']
    yu = df.loc[u,'latitude']
    yv = df.loc[v,'latitude'] 
    dfg.loc[i,'geoDistance'] =  (((xu-xv)**2)  + ((yu-yv)**2))**0.5
    
    dfg.loc[i,'reviewsDiff'] = abs(df.loc[u,'N_reviews'] - df.loc[v,'N_reviews'])  
    dfg.loc[i,'scoreDiff'] = abs(df.loc[u,'score'] - df.loc[v,'score'])
    dfg.loc[i,'priceDiff'] = abs(df.loc[u,'price'] - df.loc[v,'price'])
    dfg.loc[i,'reviewsSum'] = df.loc[u,'N_reviews'] + df.loc[v,'N_reviews']
    dfg.loc[i,'scoreSum'] = df.loc[u,'score'] + df.loc[v,'score']
    dfg.loc[i,'priceSum'] = df.loc[u,'price'] + df.loc[v,'price']
    
    
    specialDietsu = df.loc[u,'specialDiets']
    specialDietsv = df.loc[v,'specialDiets']
    if specialDietsu != 0 and specialDietsv != 0:
        dfg.loc[i,'specialDiets'] = len(set(specialDietsu.split(', ')).intersection(set(specialDietsv.split(', '))))
    else: dfg.loc[i,'specialDiets'] = 0
    
    cuisinesu = df.loc[u,'cuisines']
    cuisinesv = df.loc[v,'cuisines']
    if cuisinesu != 0 and cuisinesv != 0:
        dfg.loc[i,'cuisines'] = len(set(cuisinesu.split(', ')).intersection(set(cuisinesv.split(', '))))
    else: dfg.loc[i,'cuisines'] = 0
    
    for boolean in ['menu','covidMeasure','travellersChoice','claimed']:
        dfg.loc[i,boolean] = df.loc[u,boolean] + df.loc[v,boolean]
    
    
    

In [ ]:
    cuisinesv = df.loc[v,'cuisines']
    if cuisinesu != 0 and cuisinesv != 0:
        dfg.loc[i,'cuisines'] = len(set(cuisinesu.split(', ')).interception(set(cuisinesv.split(', '))))
    else: dfg.loc[i,'cuisines'] = 0

In [26]:
df.columns

Index(['index', 'position', 'name', 'city', 'menu', 'link', 'score',
       'N_reviews', 'price', 'Name', 'Province', 'City', 'claimed',
       'covidMeasure', 'travellersChoice', 'latitude', 'longitude', 'cuisines',
       'meals', 'specialDiets', 'positionlink', 'province', 'Link',
       'provinceOK', 'louvain', 'kmeans', 'async_fluid_7', 'multicore',
       'demon'],
      dtype='object')

In [33]:
df[df.specialDiets.isna()]

,index,position,name,city,menu,link,score,N_reviews,price,Name,...,specialDiets,positionlink,province,Link,provinceOK,louvain,kmeans,async_fluid_7,multicore,demon
8,238,1719,Osteria La Porta del Chianti,Impruneta,False,/Restaurant_Review-g635637-d2359383-Reviews-Os...,4.0,291,2.5,Osteria La Porta del Chianti,...,NaN,t4A_https://maps.google.com/maps?saddr=&daddr=...,florence,/Restaurant_Review-g635637-d2359383-Reviews-Os...,1,8.0,0.0,6.0,2.0,58
9,275,1480,Ristorante Carmen Bay,Capalbio,False,/Restaurant_Review-g1015997-d1894478-Reviews-R...,3.0,239,2.5,Ristorante Carmen Bay,...,NaN,GIM_https://maps.google.com/maps?saddr=&daddr=...,grosseto,/Restaurant_Review-g1015997-d1894478-Reviews-R...,1,8.0,6.0,4.0,0.0,"58, 160"
10,278,1493,Il Ginepro Coccolone,Capalbio,False,/Restaurant_Review-g1015997-d4731709-Reviews-I...,3.0,74,2.5,Il Ginepro Coccolone,...,NaN,XB0_https://maps.google.com/maps?saddr=&daddr=...,grosseto,/Restaurant_Review-g1015997-d4731709-Reviews-I...,1,8.0,6.0,4.0,2.0,"58, 90"
25,415,158,Ristorante Da Palmira - Monte a Pescia,Monte a Pescia,True,/Restaurant_Review-g1186342-d1902898-Reviews-R...,4.0,279,2.5,Ristorante Da Palmira - Monte a Pescia,...,NaN,iW3_https://maps.google.com/maps?saddr=&daddr=...,pistoia,/Restaurant_Review-g1186342-d1902898-Reviews-R...,1,8.0,3.0,3.0,0.0,"58, 90"
26,421,155,Trattoria Pizzeria DA NERONE,Pescia,False,/Restaurant_Review-g1016807-d2437637-Reviews-T...,4.0,215,2.5,Trattoria Pizzeria DA NERONE,...,NaN,bIw_https://maps.google.com/maps?saddr=&daddr=...,pistoia,/Restaurant_Review-g1016807-d2437637-Reviews-T...,1,8.0,3.0,3.0,1.0,"58, 90"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14998,11035,1755,Bar di Sotto,Elba Island,False,/Restaurant_Review-g194881-d21296721-Reviews-B...,5.0,1,NaN,Bar di Sotto,...,NaN,Rxn_https://maps.google.com/maps?saddr=&daddr=...,livorno,NaN,1,10.0,9.0,2.0,3.0,NaN
14999,1449,1328,Terrazza,Follonica,False,/Restaurant_Review-g194764-d20989093-Reviews-T...,5.0,1,2.5,Terrazza,...,NaN,cpI_https://maps.google.com/maps?saddr=&daddr=...,grosseto,/Restaurant_Review-g194764-d20989093-Reviews-T...,1,5.0,5.0,6.0,0.0,NaN
15000,1681,1230,Beach Bar,Follonica,False,/Restaurant_Review-g194764-d20989105-Reviews-B...,5.0,2,2.5,Beach Bar,...,NaN,DpJ_https://maps.google.com/maps?saddr=&daddr=...,grosseto,/Restaurant_Review-g194764-d20989105-Reviews-B...,1,5.0,5.0,6.0,0.0,NaN
15001,3808,696,Kebab Ali Baba,Massa,False,/Restaurant_Review-g660766-d15862036-Reviews-K...,4.0,2,NaN,Kebab Ali Baba,...,NaN,oC8_https://maps.google.com/maps?saddr=&daddr=...,massacarrara,/Restaurant_Review-g660766-d15862036-Reviews-K...,1,3.0,2.0,6.0,0.0,NaN


In [69]:
dfg.to_csv('../data/linkPrediction.csv')

### Train Test split

In [3]:

dfg = pd.read_csv('../data/linkPrediction.csv',index_col=0)



In [8]:
from sklearn.model_selection import train_test_split, cross_val_score 

from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [5]:
dfg['connected'] = 1
dfg.loc[dfg['weight'] ==0,'connected'] = 0
dfg.connected.value_counts()

0    136646
1    136646
Name: connected, dtype: int64

In [6]:
dfg.columns

Index(['u', 'v', 'weight', 'province', 'city', 'geoDistance', 'reviewsDiff',
       'scoreDiff', 'priceDiff', 'reviewsSum', 'scoreSum', 'priceSum',
       'specialDiets', 'cuisines', 'menu', 'covidMeasure', 'travellersChoice',
       'claimed', 'connected'],
      dtype='object')

In [9]:
attributes = ['province', 'city', 'geoDistance', 'reviewsDiff',
       'scoreDiff', 'priceDiff', 'reviewsSum', 'scoreSum', 'priceSum',
       'specialDiets', 'cuisines', 'menu', 'covidMeasure', 'travellersChoice',
       'claimed'] 
target =  'connected'

X = dfg[attributes].values
y = dfg[target].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100, stratify=y)
    

### Random Forest

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [13]:
clf = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=10, 
                             min_samples_leaf=3, min_weight_fraction_leaf=0.0, max_features='auto', random_state=0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.762745767673318
F1-score [0.77761518 0.74574543]
              precision    recall  f1-score   support

           0       0.73      0.83      0.78     40994
           1       0.80      0.70      0.75     40994

    accuracy                           0.76     81988
   macro avg       0.77      0.76      0.76     81988
weighted avg       0.77      0.76      0.76     81988



### AdaBoost

In [14]:
from sklearn.ensemble import AdaBoostClassifier

In [15]:
clf = AdaBoostClassifier(base_estimator=RandomForestClassifier(n_estimators=100), n_estimators=400, random_state=0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.7599892667219593
F1-score [0.77247711 0.746051  ]
              precision    recall  f1-score   support

           0       0.73      0.81      0.77     40994
           1       0.79      0.71      0.75     40994

    accuracy                           0.76     81988
   macro avg       0.76      0.76      0.76     81988
weighted avg       0.76      0.76      0.76     81988



### Bagging

In [16]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier

In [17]:
clf = BaggingClassifier(base_estimator=None, n_estimators=100, random_state=0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.7540981607064449
F1-score [0.76468048 0.74251925]
              precision    recall  f1-score   support

           0       0.73      0.80      0.76     40994
           1       0.78      0.71      0.74     40994

    accuracy                           0.75     81988
   macro avg       0.76      0.75      0.75     81988
weighted avg       0.76      0.75      0.75     81988

